# Comminity C 생성
- 라벨링된 지역별 user_id --> inner_id 변환
- 00_word.csv에서 label_result.csv의 user_id와 매칭되는 inner_id로 변환하기

In [35]:
import pandas as pd
import numpy as np

In [36]:
labeled_df = pd.read_csv(r'C:\Users\haech\label_result.csv')
labeled_df['region'].value_counts()

수도    1283
경상     837
충청     319
강원     224
전라     129
Name: region, dtype: int64

In [37]:
labeled_df['gw'] = labeled_df['region'].isin(['강원'])
labeled_df['cc'] = labeled_df['region'].isin(['충청'])
labeled_df['jl'] = labeled_df['region'].isin(['전라'])
labeled_df['sd'] = labeled_df['region'].isin(['수도'])
labeled_df['gs'] = labeled_df['region'].isin(['경상'])

In [38]:
labeled_df

,region,insta_id,gw,cc,jl,sd,gs
0,수도,0.21_se,False,False,False,True,False
1,수도,00_msssss,False,False,False,True,False
2,수도,00_wlwl,False,False,False,True,False
3,충청,01.su_jin,False,True,False,False,False
4,충청,0104ayoung,False,True,False,False,False
...,...,...,...,...,...,...,...
2787,수도,zumba.hara,False,False,False,True,False
2788,수도,zumba_honey,False,False,False,True,False
2789,수도,zusainbolt,False,False,False,True,False
2790,수도,zzang__mat,False,False,False,True,False


In [39]:
gw_index = labeled_df.index[labeled_df['gw'] == True].tolist()
cc_index = labeled_df.index[labeled_df['cc'] == True].tolist()
jl_index = labeled_df.index[labeled_df['jl'] == True].tolist()
sd_index = labeled_df.index[labeled_df['sd'] == True].tolist()
gs_index = labeled_df.index[labeled_df['gs'] == True].tolist()

In [40]:
print(len(gw_index), len(cc_index), len(jl_index), len(sd_index), len(gs_index))

224 319 129 1283 837


In [41]:
labeled_ls = labeled_df['insta_id'].tolist()

In [42]:
# 지역별 labeling된 user_id 리스트 생성
def extract_id(index, labeled_ls):
    id_ls = []
    for i, v in enumerate(labeled_ls):
        for j in index:
            if i == j:
                id_ls.append(v)
    return id_ls

In [43]:
gw_userid = extract_id(gw_index, labeled_ls)
cc_userid = extract_id(cc_index, labeled_ls)
jl_userid = extract_id(jl_index, labeled_ls)
sd_userid = extract_id(sd_index, labeled_ls)
gs_userid = extract_id(gs_index, labeled_ls)

### 모든 지역 word.csv합치기
- 모든 지역의 user_id와 inner_id를 추출하기 위함

In [44]:
df1 = pd.read_csv(r'C:\Users\haech\gw_word.csv')
df2 = pd.read_csv(r'C:\Users\haech\cc_word.csv')
df3 = pd.read_csv(r'C:\Users\haech\gs_word.csv')
df4 = pd.read_csv(r'C:\Users\haech\jl_word.csv')
df5 = pd.read_csv(r'C:\Users\haech\sd_word.csv')

In [45]:
result = df1.append(df2)
result = result.append(df3)
result = result.append(df4)
result = result.append(df5)

In [46]:
result.reset_index(inplace=True)

In [47]:
df = result.drop('index', axis=1) # 전체 지역 합친 거

In [48]:
id_df = df[['insta_id', 'inner_id']]

### 각 지역별 labeling된 userid를 innerid로 변환하기
- {insta_id : inner_id} 사전생성 후, 매칭되는 userid를 innerid로 변환

In [22]:
# 전체 지역의 {insta_id : inner_id} 사전생성

def create_id_dic(id_df):
    id_dic = {}
    for insta, inner in zip(id_df['insta_id'], id_df['inner_id']):
        if insta in id_dic.keys() or inner in id_dic.values():
            pass
        else:
            id_dic[insta] = inner
    return id_dic

In [23]:
all_id_dic = create_id_dic(id_df)
len(all_id_dic)

5589

In [24]:
# 각 지역별 labeling된 userid를 innerid로 변환하기

def user_to_inner(all_id_dic, userid_ls):
    innerid = []
    for (key, elem) in all_id_dic.items():
        for i in userid_ls:
            if i == key:
                innerid.append(elem)
    return innerid

In [25]:
# labeling된 유저들의 지역별 inner_id

gw_innerid = user_to_inner(all_id_dic, gw_userid)
cc_innerid = user_to_inner(all_id_dic, cc_userid)
jl_innerid = user_to_inner(all_id_dic, jl_userid)
sd_innerid = user_to_inner(all_id_dic, sd_userid)
gs_innerid = user_to_inner(all_id_dic, gs_userid)

In [26]:
type(jl_innerid[0])

int

In [27]:
print(len(gw_innerid), len(cc_innerid), len(jl_innerid), len(sd_innerid), len(gs_innerid))

224 319 129 1281 836


### 각 지역별 center user로부터 수집한 팔로워/팔로잉 id목록에서 labeling된 id만 남기기
- 예시 . 강원도

In [28]:
jl_df = pd.read_csv(r"C:\Users\haech\jl_2.csv")

In [29]:
jl_df['t_f'] = np.nan
jl_df

,start,end,t_f
0,31787118935,16574483307,NaN
1,16574483307,2931147151,NaN
2,16574483307,16360242589,NaN
3,16574483307,1999007700,NaN
4,16574483307,2371238066,NaN
...,...,...,...
10180,201464065,390379796,NaN
10181,201464065,6265992188,NaN
10182,201464065,1754242539,NaN
10183,201464065,1712946934,NaN


In [30]:
for i,v in enumerate(jl_df['start']):
    for j in jl_innerid:
        if v == j:
            jl_df['t_f'][i] = 'T1'

for i,v in enumerate(jl_df['end']):            
    for j in jl_innerid:
        if v == j:
            jl_df['t_f'][i] = 'T2'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ 

In [33]:
jl_df = jl_df.dropna()
jl_df.drop(['t_f'], axis='columns', inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [34]:
jl_df # 결과

,start,end
84,39245477197,2212358887
854,5372467479,20911270630
1403,20911270630,5372467479
1548,4520758326,4044075687
1902,20911270630,2080289377
3397,5367931874,7185274605
3402,32688567176,7686380442
3706,6048680223,5732244865
4339,6581581630,7364826823
5526,7364826823,6581581630
